# Test

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np

from itertools import permutations, product, combinations

from scipy.optimize import minimize

In [3]:
from mppm.optimization.probabilities.team_prob import *
from mppm.optimization.probabilities.group_probabilities import GroupProbabilities
from mppm.optimization.probabilities.octavo_probabilities import OctavoProbabilities
from mppm.optimization.probabilities.quarter_probabilities import QuarterProbabilities
from mppm.optimization.probabilities.semi_probabilities import SemiProbabilities
from mppm.optimization.probabilities.final_probabilities import FinalProbabilities

In [4]:
from mppm.porra.outcome import Outcome, Result
from mppm.porra.helper import GROUPS
from mppm.qatar.progression import QatarProgression
from mppm.qatar.scoring import QatarScoring

In [5]:
GroupA = GroupProbabilities(
    QATAR,
    ECUADOR,
    SENEGAL,
    NETHERLANDS
)

In [6]:
GroupB = GroupProbabilities(
    ENGLAND,
    IRAN,
    USA,
    WALES
)

In [7]:
GroupC = GroupProbabilities(
    ARGENTINA,
    SAUDI_ARABIA,
    MEXICO,
    POLAND
)

In [8]:
GroupD = GroupProbabilities(
    FRANCE,
    DENMARK,
    TUNISIA,
    AUSTRALIA
)

In [9]:
GroupE = GroupProbabilities(
    SPAIN,
    GERMANY,
    JAPAN,
    COSTA_RICA
)

In [10]:
GroupF = GroupProbabilities(
    BELGIUM,
    CANADA,
    MOROCCO,
    CROATIA
)

In [11]:
GroupG = GroupProbabilities(
    BRAZIL,
    SERBIA,
    SWITZERLAND,
    CAMEROON
)

In [12]:
GroupH = GroupProbabilities(
    PORTUGAL,
    GHANA,
    URUGUAY,
    SOUTH_KOREA
)

In [13]:
Oct1 = OctavoProbabilities(
    GroupA,
    GroupB
)

In [14]:
Oct2 = OctavoProbabilities(
    GroupC,
    GroupD
)

In [15]:
Oct3 = OctavoProbabilities(
    GroupE,
    GroupF
)

In [16]:
Oct4 = OctavoProbabilities(
    GroupG,
    GroupH
)

In [17]:
QF1 = QuarterProbabilities(
    Oct1,
    Oct2
)

In [18]:
QF2 = QuarterProbabilities(
    Oct3,
    Oct4
)

In [19]:
SF = SemiProbabilities(QF1, QF2)

In [20]:
F = FinalProbabilities(SF)

In [21]:
RedA = {key: val for key, val in GroupA.group_probabilities().items() if val > 0.105}
RedB = {key: val for key, val in GroupB.group_probabilities().items() if val > 0.105}
RedC = {key: val for key, val in GroupC.group_probabilities().items() if val > 0.105}
RedD = {key: val for key, val in GroupD.group_probabilities().items() if val > 0.105}
RedE = {key: val for key, val in GroupE.group_probabilities().items() if val > 0.105}
RedF = {key: val for key, val in GroupF.group_probabilities().items() if val > 0.105}
RedG = {key: val for key, val in GroupG.group_probabilities().items() if val > 0.105}
RedH = {key: val for key, val in GroupH.group_probabilities().items() if val > 0.105}

In [22]:
GR = [x for x in product(
    RedA.keys(),
    RedB.keys(),
    RedC.keys(),
    RedD.keys(),
    RedE.keys(),
    RedF.keys(),
    RedG.keys(),
    RedH.keys()
)]

In [67]:
len(GR)

1536

In [24]:
OCT = [x for x in product(range(0, 2), repeat=8)]

In [25]:
QUART = [x for x in product(range(0, 2), repeat=4)]

In [26]:
SEMI = [x for x in product(range(0, 2), repeat=2)]

In [27]:
FIN = [x for x in product(range(0, 2), repeat=1)]

In [28]:
ALL = [Result(
        groups={letter: list(res) for letter, res in zip(GROUPS, x[0])},
        octavos=list(x[1]),
        quarter=list(x[2]),
        semi=list(x[3]),
        final=list(x[4])
    ) for x in product(
    product(
        RedA.keys(),
        RedB.keys(),
        RedC.keys(),
        RedD.keys(),
        RedE.keys(),
        RedF.keys(),
        RedG.keys(),
        RedH.keys()
    ),
    product(range(0, 2), repeat=8),
    product(range(0, 2), repeat=4),
    product(range(0, 2), repeat=2),
    product(range(0, 2), repeat=2)
)]

In [30]:
len(ALL)

33554432

In [23]:
JUST_GROUPS = [Result(
        groups={letter: list(res) for letter, res in zip(GROUPS, x)},
        octavos=[0]*8,
        quarter=[0]*4,
        semi=[0]*2,
        final=[0]*2
    ) for x in product(
        RedA.keys(),
        RedB.keys(),
        RedC.keys(),
        RedD.keys(),
        RedE.keys(),
        RedF.keys(),
        RedG.keys(),
        RedH.keys()
    )]

In [24]:
len(JUST_GROUPS)

1536

In [70]:
ALL_RED = [RedA, RedB, RedC, RedD, RedE, RedF, RedG, RedH]

Prob = {
    x: np.array([group[key] for group, key in zip(ALL_RED, x)]).prod() for x in product(
        RedA.keys(),
        RedB.keys(),
        RedC.keys(),
        RedD.keys(),
        RedE.keys(),
        RedF.keys(),
        RedG.keys(),
        RedH.keys()
    )
}

In [71]:
res = {}
for bet in JUST_GROUPS:
    res[bet] = [
        Outcome(QatarProgression, bet, final, QatarScoring).get_group_points() for final in JUST_GROUPS
    ]

In [72]:
pr = np.array([pr for pr in Prob.values()])

expected = np.array([
    np.multiply(pr, np.array(points)) for points in res.values()
])

In [73]:
import pandas as pd

In [74]:
df = pd.DataFrame(expected)

In [75]:
max_score = {}

for i, row in df.iterrows():
    max_score[i] = df.apply(lambda x: np.maximum(row, x), axis=1).sum(axis=1).max()

In [77]:
pd.DataFrame.from_dict(max_score, orient="index").sort_values(by=0)

,0
1079,0.317002
887,0.317017
1078,0.317039
886,0.317055
1077,0.317081
...,...
461,0.325792
775,0.325796
652,0.325796
460,0.325799


In [80]:
df.apply(lambda x: np.maximum(df.loc[0], x), axis=1).sum(axis=1)

0       0.294835
1       0.306026
2       0.306100
3       0.306167
4       0.309423
          ...   
1531    0.317571
1532    0.313601
1533    0.314426
1534    0.314477
1535    0.315908
Length: 1536, dtype: float64

In [97]:
JUST_GROUPS[652].groups

{'A': [3, 1, 2, 0],
 'B': [0, 3, 2, 1],
 'C': [0, 3, 2, 1],
 'D': [0, 1, 3, 2],
 'E': [1, 0, 2, 3],
 'F': [0, 3, 2, 1],
 'G': [0, 1, 2, 3],
 'H': [0, 2, 1, 3]}

In [82]:
JUST_GROUPS[967].groups

{'A': [3, 2, 1, 0],
 'B': [0, 3, 1, 2],
 'C': [0, 3, 2, 1],
 'D': [0, 1, 2, 3],
 'E': [0, 1, 2, 3],
 'F': [0, 3, 2, 1],
 'G': [0, 2, 1, 3],
 'H': [0, 3, 1, 2]}

In [25]:
pilli = JUST_GROUPS[967]
pablo = JUST_GROUPS[460]

In [101]:
oct1_prob  =Oct1.get_cond_prob()
oct2_prob  =Oct2.get_cond_prob()
oct3_prob  =Oct3.get_cond_prob()
oct4_prob  =Oct4.get_cond_prob()

In [111]:
print(oct1_prob[(pilli.groups["A"][0], pilli.groups["B"][1], 0)])
print(oct1_prob[(pilli.groups["A"][0], pilli.groups["B"][1], 1)])
print(oct1_prob[(pablo.groups["A"][0], pablo.groups["B"][1], 0)])
print(oct1_prob[(pablo.groups["A"][0], pablo.groups["B"][1], 1)])

0.6804911472631622
0.3197859302167745
0.6804911472631622
0.3197859302167745


In [112]:
print(oct1_prob[(pilli.groups["A"][1], pilli.groups["B"][0], 0)])
print(oct1_prob[(pilli.groups["A"][1], pilli.groups["B"][0], 1)])
print(oct1_prob[(pablo.groups["A"][1], pablo.groups["B"][0], 0)])
print(oct1_prob[(pablo.groups["A"][1], pablo.groups["B"][0], 1)])

0.31482848661343926
0.6855937659352296
0.2819245683350071
0.7184328087247658


In [113]:
print(oct2_prob[(pilli.groups["C"][0], pilli.groups["D"][1], 0)])
print(oct2_prob[(pilli.groups["C"][0], pilli.groups["D"][1], 1)])
print(oct2_prob[(pablo.groups["C"][0], pablo.groups["D"][1], 0)])
print(oct2_prob[(pablo.groups["C"][0], pablo.groups["D"][1], 1)])

0.6677325235287821
0.3322674478958121
0.6677325235287821
0.3322674478958121


In [114]:
print(oct2_prob[(pilli.groups["C"][1], pilli.groups["D"][0], 0)])
print(oct2_prob[(pilli.groups["C"][1], pilli.groups["D"][0], 1)])
print(oct2_prob[(pablo.groups["C"][1], pablo.groups["D"][0], 0)])
print(oct2_prob[(pablo.groups["C"][1], pablo.groups["D"][0], 1)])

0.25858627035693776
0.7414137233389706
0.25858627035693776
0.7414137233389706


In [115]:
print(oct3_prob[(pilli.groups["E"][0], pilli.groups["F"][1], 0)])
print(oct3_prob[(pilli.groups["E"][0], pilli.groups["F"][1], 1)])
print(oct3_prob[(pablo.groups["E"][0], pablo.groups["F"][1], 0)])
print(oct3_prob[(pablo.groups["E"][0], pablo.groups["F"][1], 1)])

0.6310913274043854
0.3689142053660166
0.6202398421971341
0.3797669114723015


In [116]:
print(oct3_prob[(pilli.groups["E"][1], pilli.groups["F"][0], 0)])
print(oct3_prob[(pilli.groups["E"][1], pilli.groups["F"][0], 1)])
print(oct3_prob[(pablo.groups["E"][1], pablo.groups["F"][0], 0)])
print(oct3_prob[(pablo.groups["E"][1], pablo.groups["F"][0], 1)])

0.5509841264302957
0.4490165404923934
0.550041143643308
0.4499573370985844


In [117]:
print(oct4_prob[(pilli.groups["G"][0], pilli.groups["H"][1], 0)])
print(oct4_prob[(pilli.groups["G"][0], pilli.groups["H"][1], 1)])
print(oct4_prob[(pablo.groups["G"][0], pablo.groups["H"][1], 0)])
print(oct4_prob[(pablo.groups["G"][0], pablo.groups["H"][1], 1)])

0.8380762899801646
0.161923655551594
0.7210860506137896
0.27891399232608266


In [118]:
print(oct4_prob[(pilli.groups["G"][1], pilli.groups["H"][0], 0)])
print(oct4_prob[(pilli.groups["G"][1], pilli.groups["H"][0], 1)])
print(oct4_prob[(pablo.groups["G"][1], pablo.groups["H"][0], 0)])
print(oct4_prob[(pablo.groups["G"][1], pablo.groups["H"][0], 1)])

0.3661158607048974
0.6338841362339417
0.3830697876549965
0.6169301813706728


In [41]:
qf1_prob = QF1.get_cond_prob()
qf2_prob = QF2.get_cond_prob()

In [38]:
print(qf1_prob[(3, 0, 0)])
print(qf1_prob[(3, 0, 1)])
print(qf1_prob[(3, 5, 0)])
print(qf1_prob[(3, 5, 1)])

0.3975604486505789
0.6025613840326187
0.60002193103271
0.4000360703245891


In [40]:
print(qf1_prob[(4, 4, 0)])
print(qf1_prob[(4, 4, 1)])
print(qf1_prob[(4, 4, 0)])
print(qf1_prob[(4, 4, 1)])

0.478868659119723
0.5213841397917961
0.478868659119723
0.5213841397917961


In [42]:
print(qf2_prob[(4, 4, 0)])
print(qf2_prob[(4, 4, 1)])

0.5300763683923103
0.4699095091655724
